In [1]:
##############
# Import libs
##############

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import re

import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
print(sys.stdout.encoding)

UTF-8


In [8]:
###########
# Settings
###########

INPUT_FILE = "data/Otvety.txt"
OUTPUT_FILE = 'output/prepared_answers.txt'
SENTENCE_LIMIT = 200000

In [18]:
# Text Overview

with open("data/Otvety.txt", "r", encoding='utf-8') as f:
    line_gen = [next(f) for x in range(15)]
    for line in line_gen:
        print(line)
        



---

вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . 

хомячка.... 

мужика, йопаря, собачку и 50 кошек))). 

Общение !). 

паучка. 

Да пол мне бы памыть!<br>А таг то ни чо. Типа ни каво!. 

я тут вообще что бы пообщаться..... 

А мне советовали сиси завести...))). 

Ну, слава богу, мужика завести ещё не советовали))) А вот сватать к кому только не сватали). 

мне тут советовали завести любовника, мужа и много кошек )))<br>приветик ))). 

---

Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) . 


меня вобще прикалывает эта тема :). 



In [42]:
###############
# Modules
###############


def extract_answers(input_file,
                    output_file
                   ):

    """Extracts and write answers to distinct file"""
    
    question = None
    written = False

    with open(input_file, "w", encoding='utf-8') as fout:

        with open(output_file, "r", encoding="utf-8") as fin:

            for line in tqdm(fin): 
                
                # 1-st line after '---' == question
                # lines to '---' == answers

                if line.startswith("---"):
                    written = False
                    continue

                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                    written = True
                    question = None
                    continue

                if not written:
                    question = line.strip()
                    continue
    
    
    
def preprocess_txt(line, exclude=exclude, morpher=morpher, sw=sw):
    
    """Light text preprocessing and morpher """
    
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls


def get_sentences(input_file, len_limit=100000):
    
    """Process text. extract sentense tokens with morph processor"""
    
    # Preprocess for models fitting

    sentences = []

    morpher = MorphAnalyzer()
    sw = set(get_stop_words("ru"))
    exclude = set(string.punctuation)
    c = 0

    with open(input_file, "r", encoding='utf-8') as fin:
        for line in tqdm(fin):
            spls = preprocess_txt(line, exclude, morpher, sw)
            sentences.append(spls)
            c += 1
            if c > len_limit:
                break
                
    return sentences, exclude, morpher, sw


def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(50)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [40]:
assert False  #  Set True to prepare data if not exists

extract_answers(INPUT_FILE, OUTPUT_FILE)

AssertionError: 

In [41]:
sentences, exclude, morpher, sw = get_sentences(INPUT_FILE)

In [43]:
sentences = [i for i in sentences if len(i) > 2]

In [44]:
sentences[20]

['научно',
 'производственный',
 'предприятие',
 'итэлмавыпускать',
 'счётчик',
 'электроника',
 'немецкий',
 'технология']

In [45]:
modelW2V = Word2Vec(sentences=sentences, size=50, window=5, min_count=1)

In [46]:
modelFT = FastText(sentences=sentences, size=50, min_count=1, window=5)

In [48]:
w2v_index = annoy.AnnoyIndex(50 ,'angular')
ft_index = annoy.AnnoyIndex(50 ,'angular')

index_map = {}
counter = 0

with open(OUTPUT_FILE, "r", encoding='utf-8') as f:
    for line in tqdm(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(50)
        vector_ft = np.zeros(50)
        for word in question:
            if word in modelW2V:
                vector_w2v += modelW2V[word]
                n_w2v += 1
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > SENTENCE_LIMIT:
            break

w2v_index.build(10)
ft_index.build(10)

True

In [49]:
w2v_index.get_nns_by_vector(np.zeros(50), 3)

[4052, 7949, 12846]

In [50]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(50)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [51]:
TEXT = "Когда нейроны в нервной системе протаптывают тропинки?"

In [52]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['<p> Соматическая нервная система подчинена воле человека, т.е. захотел сел, захотел руку поднял. Она регулирует работу скелетных мышц, ее центры находятся в коре больших полушарий(КБП). </p>. \n',
 'Только не рабы, а продукты.. \n',
 'Нет не повредит. Только приведёт в порядок файловую систему, при это ничего не удаляется. Если это конечно, Акронис, а не палёная троянская версия :))). \n']

In [53]:
get_response(TEXT, ft_index, modelFT, index_map)

['где? в мозгу... \n',
 'http://bikes.ironhorse.ru/archives/36. \n',
 'раньше мечтали о свечном заводике, а теперь о нефтяных вышках... разузнайте почту владельца нефтяной вышки и обсудите с ним варианты купли-продажи. или дайте объявления на все возможные телевизионные каналы. (особое внимание РБК) Выгоднее будет построить нефтеперерабатывающий завод - быстрее окупится.. \n']

In [54]:
# Save results

w2v_index.save('output/w2v_idx')
ft_index.save('output/ft_idx')

with open('output/index_map.pkl', 'wb') as f:
    pickle.dump(index_map, f)
    